# Models

In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.multiclass import OneVsRestClassifier
import string
import ast
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read in data
imdb_movie = pd.read_csv('data/imdb_multilabel.csv')

In [3]:
imdb_movie.head()

,Unnamed: 0,title,imdb_id,topRank,bottomRank,metaScore,plot,rating,ratingCount,reviewCount,runningTimeInMinutes,userRatingCount,userScore,year,all_genre,genre,plot_list,genreCount,genre_code,all_genre_encode
0,0,"I, Tonya",tt5580036,930.0,17643.0,77.0,From the proverbial wrong side of the tracks i...,7.6,67667.0,46,120.0,235,7.8,2017.0,"['Biography', 'Comedy', 'Drama', 'Sport']",sport,"['From', 'the', 'proverbial', 'wrong', 'side',...",4,0,[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
1,1,Cars 3,tt3606752,2256.0,11547.0,59.0,Blindsided by a new generation of blazing-fast...,6.8,41896.0,41,102.0,232,6.9,2017.0,"['Animation', 'Adventure', 'Comedy', 'Family',...",sport,"['Blindsided', 'by', 'a', 'new', 'generation',...",5,0,[1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. ...
2,2,Creed,tt3076658,847.0,17840.0,82.0,Adonis Johnson is the son of the famous boxing...,7.6,193206.0,42,133.0,614,8.0,2015.0,"['Drama', 'Sport']",sport,"['Adonis', 'Johnson', 'is', 'the', 'son', 'of'...",2,0,[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
3,3,Battle of the Sexes,tt4622512,2303.0,11228.0,73.0,In the wake of the sexual revolution and the r...,6.8,27960.0,46,121.0,102,6.3,2017.0,"['Biography', 'Comedy', 'Drama', 'Sport']",sport,"['In', 'the', 'wake', 'of', 'the', 'sexual', '...",4,0,[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
4,4,Borg McEnroe,tt5727282,NaN,12891.0,57.0,The story of the 1980s tennis rivalry between ...,7.0,9800.0,13,107.0,0,NaN,2017.0,"['Biography', 'Drama', 'Sport']",sport,"['The', 'story', 'of', 'the', '1980s', 'tennis...",3,0,[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...


### Encoding

#### single-label encode

In [4]:
genre_dict = dict(zip(imdb_movie.genre.unique(), range(20)))

imdb_movie['genre_code'] = imdb_movie.genre.replace(genre_dict).values

In [5]:
imdb_movie.head()

,Unnamed: 0,title,imdb_id,topRank,bottomRank,metaScore,plot,rating,ratingCount,reviewCount,runningTimeInMinutes,userRatingCount,userScore,year,all_genre,genre,plot_list,genreCount,genre_code,all_genre_encode
0,0,"I, Tonya",tt5580036,930.0,17643.0,77.0,From the proverbial wrong side of the tracks i...,7.6,67667.0,46,120.0,235,7.8,2017.0,"['Biography', 'Comedy', 'Drama', 'Sport']",sport,"['From', 'the', 'proverbial', 'wrong', 'side',...",4,0,[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
1,1,Cars 3,tt3606752,2256.0,11547.0,59.0,Blindsided by a new generation of blazing-fast...,6.8,41896.0,41,102.0,232,6.9,2017.0,"['Animation', 'Adventure', 'Comedy', 'Family',...",sport,"['Blindsided', 'by', 'a', 'new', 'generation',...",5,0,[1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. ...
2,2,Creed,tt3076658,847.0,17840.0,82.0,Adonis Johnson is the son of the famous boxing...,7.6,193206.0,42,133.0,614,8.0,2015.0,"['Drama', 'Sport']",sport,"['Adonis', 'Johnson', 'is', 'the', 'son', 'of'...",2,0,[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
3,3,Battle of the Sexes,tt4622512,2303.0,11228.0,73.0,In the wake of the sexual revolution and the r...,6.8,27960.0,46,121.0,102,6.3,2017.0,"['Biography', 'Comedy', 'Drama', 'Sport']",sport,"['In', 'the', 'wake', 'of', 'the', 'sexual', '...",4,0,[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
4,4,Borg McEnroe,tt5727282,NaN,12891.0,57.0,The story of the 1980s tennis rivalry between ...,7.0,9800.0,13,107.0,0,NaN,2017.0,"['Biography', 'Drama', 'Sport']",sport,"['The', 'story', 'of', 'the', '1980s', 'tennis...",3,0,[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...


In [6]:
genre_dict['sci-fi'] = 17

#### multi-label encode as an array

In [7]:
def multi_label_encoder(all_genre_list):
    """ This function takes a list of genre with a dictionary that keeps track of the index of the genre
    INPUTS
    ------
    all_genre_list: list of genres
    genre_dict: dictionary of indexs
    
    OUTPUTS
    -------
    np array in {0, 1}
    """
    encode = np.zeros(20)
    all_genre_list = ast.literal_eval(all_genre_list)
    for genre in all_genre_list:
        if genre.lower() in genre_dict:
            encode[genre_dict[genre.lower()]] = 1
    return list(encode)

In [8]:
imdb_movie['all_genre_encode'] = imdb_movie['all_genre'].apply(multi_label_encoder)

In [9]:
imdb_movie['all_genre_encode'].iloc[0]

[1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0]

In [10]:
imdb_movie['plot_list'] = imdb_movie['plot_list'].apply(ast.literal_eval)

In [11]:
imdb_movie['plot_list'].iloc[0]

['From',
 'the',
 'proverbial',
 'wrong',
 'side',
 'of',
 'the',
 'tracks',
 'in',
 'Portland,',
 'Oregon,',
 'former',
 'competitive',
 'figure',
 'skater',
 'Tonya',
 'Harding',
 'was',
 'never',
 'fully',
 'accepted',
 'in',
 'the',
 'figure',
 'skating',
 'community',
 'for',
 'not',
 'inherently',
 'being',
 'the',
 'image',
 'of',
 'grace,',
 'breeding',
 'and',
 'privilege',
 'that',
 'the',
 'community',
 'wanted',
 'to',
 'portray,',
 'despite',
 'she',
 'being',
 'naturally',
 'gifted',
 'in',
 'the',
 'sport',
 'athletically.',
 'Despite',
 'ultimately',
 'garnering',
 'some',
 'success',
 'in',
 'figure',
 'skating',
 'being',
 'national',
 'champion,',
 'a',
 'world',
 'championship',
 'medalist,',
 'an',
 'Olympian,',
 'and',
 'being',
 'the',
 'first',
 'American',
 'woman',
 'to',
 'complete',
 'a',
 'Triple',
 'Axel',
 'in',
 'competition,',
 'she',
 'is',
 'arguably',
 'best',
 'known',
 'for',
 'her',
 'association',
 'to',
 '"the',
 'incident":',
 'the',
 'leg',
 '

In [35]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(imdb_movie['plot'], 
                                                                            imdb_movie['all_genre_encode'],
                                                                            test_size = 0.2,
                                                                            random_state = 209,
                                                                            )

### Evaluation Function

In [36]:
def score(y_true, y_score, method='avg'):
    if method == 'avg':
        return np.mean(np.mean(y_true == y_score, axis=1))
    elif method == 'exact':
        return metrics.accuracy_score(y_true, y_score)
    elif method == 'recall':
        return np.mean([metrics.recall_score(y_true[:,i], y_score[:,i]) for i in range(y_score.shape[1])])
    elif method == 'precision':
        return np.mean([metrics.precision_score(y_true[:,i], y_score[:,i]) for i in range(y_score.shape[1])])

## Bag-of-Words Representation
### Naive Bayes 

In [37]:
# bag of words representation
vectorizer = CountVectorizer(stop_words='english')

In [39]:
naive_X_train = vectorizer.fit_transform(X_train)
naive_X_test = vectorizer.transform(X_test)

In [40]:
# reshape y_train and y_test
y_train = np.array(list(y_train))
y_test = np.array(list(y_test))

In [41]:
y_train.shape

(5124, 20)

In [42]:
y_train

array([[ 0.,  0.,  0., ...,  1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  1., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [43]:
nb = OneVsRestClassifier(MultinomialNB())
nb.fit(naive_X_train, y_train)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
          n_jobs=1)

In [44]:
naive_train_pred = nb.predict(naive_X_train)
naive_test_pred = nb.predict(naive_X_test)
print('Naive Bayes Accuracy on Train : {}'.format(metrics.accuracy_score(y_train, naive_train_pred)))
print('Naive Bayes Accuracy on Test : {}'.format(metrics.accuracy_score(y_test, naive_test_pred)))

Naive Bayes Accuracy on Train : 0.5915300546448088
Naive Bayes Accuracy on Test : 0.14508580343213728


In [45]:
nb.predict_proba(naive_X_train)[0]

array([  5.37012637e-33,   4.31795630e-31,   3.59365748e-27,
         2.63671879e-10,   1.11016946e-28,   1.01828361e-21,
         3.29233909e-25,   1.31202345e-21,   0.00000000e+00,
         1.39414154e-08,   1.33012783e-02,   4.52588102e-15,
         8.36423998e-01,   3.23838390e-14,   1.86421000e-18,
         3.69704282e-05,   8.66331663e-01,   9.99998836e-01,
         9.93130417e-01,   8.06854819e-08])

In [46]:
# evaluation
print('Naive Bayes Avg Accuracy on test : {}'.format(score(y_test, naive_test_pred, 'avg')))
print('Naive Bayes Exact Accuracy on test : {}'.format(score(y_test, naive_test_pred, 'exact')))
print('Naive Bayes Precision on test : {}'.format(score(y_test, naive_test_pred, 'precision')))
print('Naive Bayes Recall on test : {}'.format(score(y_test, naive_test_pred, 'recall')))

Naive Bayes Avg Accuracy on test : 0.8982059282371295
Naive Bayes Exact Accuracy on test : 0.14508580343213728
Naive Bayes Precision on test : 0.6978311158754985
Naive Bayes Recall on test : 0.4300237937094945


### Random Forests

In [47]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
rfc = OneVsRestClassifier(RandomForestClassifier())
rfc.fit(naive_X_train, y_train)
rfc_train_pred = rfc.predict(naive_X_train)
rfc_test_pred = rfc.predict(naive_X_test)
print('Random Forest Accuracy on Train : {}'.format(metrics.accuracy_score(y_train, rfc_train_pred)))
print('Random Forest Accuracy on Test : {}'.format(metrics.accuracy_score(y_test, rfc_test_pred)))

Random Forest Accuracy on Train : 0.7781030444964872
Random Forest Accuracy on Test : 0.0514820592823713


In [49]:
# evaluation
print('Random Forest Avg Accuracy on test : {}'.format(score(y_test, rfc_test_pred, 'avg')))
print('Random Forest Exact Accuracy on test : {}'.format(score(y_test, rfc_test_pred, 'exact')))
print('Random Forest Precision on test : {}'.format(score(y_test, rfc_test_pred, 'precision')))
print('Random Forest Recall on test : {}'.format(score(y_test, rfc_test_pred, 'recall')))

Random Forest Avg Accuracy on test : 0.869773790951638
Random Forest Exact Accuracy on test : 0.0514820592823713
Random Forest Precision on test : 0.7116486664026709
Random Forest Recall on test : 0.17130686888834687


## GloVE

In [26]:
with open("glove.6B.300d.txt", "rb") as lines:
    w2v = {line.decode("utf-8").split()[0]: np.array(line.split()[1:]).astype(float)
           for line in lines}

In [50]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = 300


    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in re.sub('['+string.punctuation+']', '', words.strip()) if w in self.word2vec] or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

mean_embed = MeanEmbeddingVectorizer(w2v)
glove_X_train = mean_embed.transform(X_train)
glove_X_test = mean_embed.transform(X_test)

In [51]:
glove_X_train.shape

(5124, 300)

### Random Forests

In [52]:
rf = OneVsRestClassifier(RandomForestClassifier())
rf.fit(glove_X_train, y_train)

rf_train_pred = rf.predict(glove_X_train)
rf_test_pred = rf.predict(glove_X_test)
print('Random Forest Accuracy on Train : {}'.format(metrics.accuracy_score(y_train, rf_train_pred)))
print('Random Forest Accuracy on Test : {}'.format(metrics.accuracy_score(y_test, rf_test_pred)))

Random Forest Accuracy on Train : 0.7090163934426229
Random Forest Accuracy on Test : 0.014040561622464899


In [53]:
# evaluation
print('Random Forest Avg Accuracy on test : {}'.format(score(y_test, rf_test_pred, 'avg')))
print('Random Forest Exact Accuracy on test : {}'.format(score(y_test, rf_test_pred, 'exact')))
print('Random Forest Precision on test : {}'.format(score(y_test, rf_test_pred, 'precision')))
print('Random Forest Recall on test : {}'.format(score(y_test, rf_test_pred, 'recall')))

Random Forest Avg Accuracy on test : 0.8392745709828392
Random Forest Exact Accuracy on test : 0.014040561622464899
Random Forest Precision on test : 0.17422571214883256
Random Forest Recall on test : 0.05759442981270631


### Logistic Regression

In [55]:
lr = OneVsRestClassifier(LogisticRegression())
lr.fit(glove_X_train, y_train)

lr_train_pred = lr.predict(glove_X_train)
lr_test_pred = lr.predict(glove_X_test)
print('Logistic Regression Accuracy on Train : {}'.format(metrics.accuracy_score(y_train, lr_train_pred)))
print('Logistic Regression Accuracy on Test : {}'.format(metrics.accuracy_score(y_test, lr_test_pred)))

Logistic Regression Accuracy on Train : 0.024785323965651834
Logistic Regression Accuracy on Test : 0.033541341653666144


In [56]:
# evaluation
print('Logistic Regression Avg Accuracy on test : {}'.format(score(y_test, lr_test_pred, 'avg')))
print('Logistic Regression Exact Accuracy on test : {}'.format(score(y_test, lr_test_pred, 'exact')))
print('Logistic Regression Precision on test : {}'.format(score(y_test, lr_test_pred, 'precision')))
print('Logistic Regression Recall on test : {}'.format(score(y_test, lr_test_pred, 'recall')))

Logistic Regression Avg Accuracy on test : 0.8517550702028079
Logistic Regression Exact Accuracy on test : 0.033541341653666144
Logistic Regression Precision on test : 0.13437848383500556
Logistic Regression Recall on test : 0.04199852722650322


### Neural Network

In [57]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten
from keras.models import Model, Sequential

Using TensorFlow backend.


In [58]:
# read in glove file
embeddings_index = {}
with open("glove.6B.300d.txt", "rb") as f:
    for line in f:
        values = line.decode("utf-8").split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [59]:
MAX_WORDS = 20000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 300

In [62]:
train_texts = X_train.apply(lambda x: 
                            re.sub('['+string.punctuation+']', 
                                   '', x.strip())).values
test_texts = X_test.apply(lambda x: 
                           re.sub('['+string.punctuation+']', 
                                  '', x.strip())).values

In [63]:
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index

In [64]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
X_glove_train = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [65]:
test_sequences = tokenizer.texts_to_sequences(test_texts)
X_glove_test = pad_sequences(test_sequences, 
                             maxlen=MAX_SEQUENCE_LENGTH)

In [67]:
# build embedding matrix
num_words = min(MAX_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [72]:
# model
model = Sequential()
e = Embedding(num_words, EMBEDDING_DIM, 
              weights=[embedding_matrix], 
              input_length=MAX_SEQUENCE_LENGTH, 
              trainable=False)
model.add(e)
model.add(Conv1D(128, 5, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_glove_train, y_train,
          batch_size=128,
          epochs=10)

Epoch 1/10
5124/5124 [==============================] - 16s 3ms/step - loss: 0.3913 - acc: 0.8368
Epoch 2/10
5124/5124 [==============================] - 16s 3ms/step - loss: 0.3152 - acc: 0.8696
Epoch 3/10
5124/5124 [==============================] - 15s 3ms/step - loss: 0.2584 - acc: 0.8922
Epoch 4/10
5124/5124 [==============================] - 15s 3ms/step - loss: 0.2002 - acc: 0.9189
Epoch 5/10
5124/5124 [==============================] - 18s 4ms/step - loss: 0.1405 - acc: 0.9479
Epoch 6/10
5124/5124 [==============================] - 19s 4ms/step - loss: 0.0938 - acc: 0.9710
Epoch 7/10
5124/5124 [==============================] - 19s 4ms/step - loss: 0.0608 - acc: 0.9854
Epoch 8/10
5124/5124 [==============================] - 18s 4ms/step - loss: 0.0380 - acc: 0.9932
Epoch 9/10
5124/5124 [==============================] - 19s 4ms/step - loss: 0.0278 - acc: 0.9959
Epoch 10/10
5124/5124 [==============================] - 19s 4ms/step - loss: 0.0178 - acc: 0.9982


In [73]:
# prediction
nn_test_pred = model.predict(X_glove_test)
nn_test_pred[nn_test_pred>0.5] = 1
nn_test_pred[nn_test_pred<=0.5] = 0

In [74]:
# evaluation
print('Neural Network Avg Accuracy on test : {}'.format(score(y_test, nn_test_pred, 'avg')))
print('Neural Network Exact Accuracy on test : {}'.format(score(y_test, nn_test_pred, 'exact')))
print('Neural Network Precision on test : {}'.format(score(y_test, nn_test_pred, 'precision')))
print('Neural Network Recall on test : {}'.format(score(y_test, nn_test_pred, 'recall')))

Neural Network Avg Accuracy on test : 0.8669656786271451
Neural Network Exact Accuracy on test : 0.05226209048361934
Neural Network Precision on test : 0.5231920016156562
Neural Network Recall on test : 0.3205018980845515
